Split start and end information into a separate row( easier to work with Tableau for some visualizations)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
trip_data = pd.read_csv("data/processed.csv",dtype="str")

In [3]:
trip_data.shape

(55379, 17)

In [4]:
splitted_data = pd.DataFrame(columns=['trip_id','direction','time','station_id','lat','lon','passholder_type'],dtype="str")

In [5]:
append_data = trip_data.loc[0:,["trip_id","start_time","start_station_id","start_lat","start_lon","passholder_type"]]
append_data.columns = ['trip_id','time','station_id','lat','lon','passholder_type']
append_data['direction'] = "start"

In [6]:
splitted_data = splitted_data.append(append_data,ignore_index=True)

In [7]:
append_data = trip_data.loc[0:,["trip_id","end_time","end_station_id","end_lat","end_lon","passholder_type"]]
append_data.columns = ['trip_id','time','station_id','lat','lon','passholder_type']
append_data['direction'] = "end"

splitted_data = splitted_data.append(append_data,ignore_index=True)

In [8]:
#check data for correctness
splitted_data.shape

(110758, 7)

In [9]:
trip_data.head(2)

,trip_id,duration,start_time,end_time,start_station_id,start_lat,start_lon,end_station_id,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,revenue_gain,usual_distance,usual_duration
0,1912818,180,7/7/2016 4:17,7/7/2016 4:20,3014,34.0566101,-118.23721,3014,34.0566101,-118.23721,6281,30,Round Trip,Monthly Pass,0.0,0,0
1,1919661,1980,7/7/2016 6:00,7/7/2016 6:33,3014,34.0566101,-118.23721,3014,34.0566101,-118.23721,6281,30,Round Trip,Monthly Pass,1.75,0,0


In [10]:
splitted_data.loc[[0,1,55379,55380]]

,direction,lat,lon,passholder_type,station_id,time,trip_id
0,start,34.0566101,-118.23721,Monthly Pass,3014,7/7/2016 4:17,1912818
1,start,34.0566101,-118.23721,Monthly Pass,3014,7/7/2016 6:00,1919661
55379,end,34.0566101,-118.23721,Monthly Pass,3014,7/7/2016 4:20,1912818
55380,end,34.0566101,-118.23721,Monthly Pass,3014,7/7/2016 6:33,1919661


In [11]:
trip_data.tail(1)

,trip_id,duration,start_time,end_time,start_station_id,start_lat,start_lon,end_station_id,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,revenue_gain,usual_distance,usual_duration
55378,8369650,480,9/30/2016 23:54,10/1/2016 0:02,3035,34.0484009,-118.26095,3031,34.0447006,-118.25244,6683,0,One Way,Walk-up,3.5,1009,290


In [12]:
splitted_data.loc[[55378,110757]]

,direction,lat,lon,passholder_type,station_id,time,trip_id
55378,start,34.0484009,-118.26095,Walk-up,3035,9/30/2016 23:54,8369650
110757,end,34.0447006,-118.25244,Walk-up,3031,10/1/2016 0:02,8369650


In [13]:
splitted_data.to_csv("data/processed_split_start_end.csv",index=False)

Next, we prepare data to be used in Sankey Diagram

In [41]:
grouped = trip_data.groupby(['start_station_id','end_station_id'])
sankey_data = pd.DataFrame(grouped.size().rename('counts')).reset_index()

In [42]:
#This is required to align the order of nodes appear in the Sankey Diagram
pre_rows = sankey_data.groupby('start_station_id').agg({'counts':'sum'}).reset_index().sort_values(by='counts',ascending=False)

In [43]:
pre_rows['end_station_id'] = pre_rows['start_station_id']
pre_rows['counts'] = 0

In [44]:
sankey_data = pre_rows.append(sankey_data)

In [45]:
#add station name
station_names = pd.read_csv("data/metro_station_table.csv",dtype="str")

In [46]:
station_names = station_names.filter(['Station ID','Station Name'])

In [49]:
station_names = station_names.drop_duplicates()

In [50]:
sankey_data = pd.merge(sankey_data, station_names, left_on = 'start_station_id', right_on = 'Station ID',how = 'left').filter(['start_station_id','Station Name','end_station_id','counts'])

In [51]:
sankey_data = pd.merge(sankey_data, station_names, left_on = 'end_station_id', right_on = 'Station ID',how = 'left').filter(['start_station_id','Station Name_x','end_station_id','Station Name_y','counts'])

In [52]:
sankey_data.columns = ['start_station_id','start_station_name','end_station_id','end_station_name','counts']

In [53]:
sankey_data.head()

,start_station_id,start_station_name,end_station_id,end_station_name,counts
0,3030,Main & 1st,3030,Main & 1st,0
1,3069,Broadway & 3rd,3069,Broadway & 3rd,0
2,3014,Union Station West Portal,3014,Union Station West Portal,0
3,3005,Flower & 7th,3005,Flower & 7th,0
4,3031,7th & Spring,3031,7th & Spring,0


In [54]:
sankey_data.to_csv("web/data/sankey_data.csv",index=False)